# Relatório da Atividade – Diárias: Coleta, Limpeza e Anonimização

## Objetivo
- Compreender a base `diarias_concatenadas.csv` com EDA rápida.
- Identificar e tratar dados sensíveis (PII) por meio de anonimização.
- Gerar uma saída anonimizada para demonstração segura dos resultados.

## Dataset e Pré‑requisitos
- Arquivo de entrada: `diarias_concatenadas.csv` na mesma pasta do notebook.
- Leitura com fallback de encoding (UTF‑8 → Latin‑1).

## Metodologia (passo a passo)
1) Carregamento seguro: leitura do CSV com tratamento de encoding.
2) EDA básica: dimensões, colunas, tipos, nulos por coluna e estatísticas numéricas.
3) Detecção heurística de PII:
   - Regex para e‑mail, CPF, CNPJ e telefone.
   - Heurística adicional por nomes de coluna (ex.: cpf, email, telefone, nome, matrícula etc.).
4) Anonimização:
   - Hash determinístico (SHA‑256) com sal `fatesg-ia2-salt-2025`.
   - Saída irreversível, permitindo junções internas consistentes sem expor dados pessoais.
   - Tokenização curta (12 chars) para melhor legibilidade.
5) Saída:
   - Gera `diarias_anonimizadas.csv` ao executar a célula única do notebook.

## Decisões e Justificativas
- Evitamos `dropna()` agressivo para não perder dados; a EDA evidencia nulos por coluna.
- Hashing (com sal) em vez de simples máscara, garantindo:
  - Não reversibilidade (proteção).
  - Reprodutibilidade (mesmo valor → mesmo hash).
- Padrões regex são práticos e cobrem casos comuns, mas não equivalem 100% às especificações formais (ex.: RFC de e‑mails).

## Como Executar
1) Deixe `diarias_concatenadas.csv` na mesma pasta do `diarias.ipynb`.
2) Abra o notebook e execute a célula 1.
3) Verifique o console para resumo da EDA e colunas sensíveis detectadas.
4) Confira o arquivo gerado `diarias_anonimizadas.csv`.

## Observações / Limitações
- Regex podem produzir falsos positivos/negativos em formatos atípicos.
- Heurística por nome de coluna depende da nomenclatura adotada no arquivo.
- A anonimização substitui dados sensíveis por tokens; perdas de interpretabilidade direta são esperadas.

## Próximos Passos (opcional)
- Criar um dicionário de dados e métricas de qualidade (completude, unicidade, consistência).
- Adicionar visualizações (distribuições, barras por categoria).
- Criar parametrização para salvar/ignorar a geração do CSV anonimizado conforme necessidade.

In [8]:
# EDA e anonimização de diarias_concatenadas.csv (célula única)
import pandas as pd
import re
import hashlib
from pathlib import Path

caminho = Path('diarias_concatenadas.csv')
if not caminho.exists():
    raise FileNotFoundError("Arquivo 'diarias_concatenadas.csv' não encontrado na pasta do notebook.")

try:
    df_diarias = pd.read_csv(caminho)
except UnicodeDecodeError:
    df_diarias = pd.read_csv(caminho, encoding='latin-1')

print("Dimensões:", df_diarias.shape)
print("Colunas:", list(df_diarias.columns))
print("\nTipos de dados:")
print(df_diarias.dtypes)

print("\nValores ausentes por coluna (top 20):")
print(df_diarias.isna().sum().sort_values(ascending=False).head(20))

num_cols = df_diarias.select_dtypes(include=['number']).columns
if len(num_cols):
    print("\nEstatísticas descritivas (numéricas):")
    print(df_diarias[num_cols].describe().T)

padrao_email = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
padrao_cpf   = re.compile(r"\b\d{3}\.?\d{3}\.?\d{3}-?\d{2}\b")
padrao_cnpj  = re.compile(r"\b\d{2}\.?\d{3}\.?\d{3}/?\d{4}-?\d{2}\b")
padrao_tel   = re.compile(r"\b(?:\+?\d{2,3}[\s-]?)?(?:\(?\d{2}\)?[\s-]?)?\d{4,5}[\s-]?\d{4}\b")

pii_cols = set()
for col in df_diarias.columns:
    amostra = df_diarias[col].dropna().astype(str).head(200).str[:200]
    texto_join = " ".join(amostra.tolist())
    if (padrao_email.search(texto_join) or padrao_cpf.search(texto_join) or
        padrao_cnpj.search(texto_join) or padrao_tel.search(texto_join)):
        pii_cols.add(col)

sugestivas = {'cpf','cnpj','email','e-mail','mail','telefone','celular','nome','name','rg','matricula','id_funcionario','id_pessoa'}
for col in df_diarias.columns:
    if any(p in col.lower() for p in sugestivas):
        pii_cols.add(col)

print("\nColunas sensíveis detectadas (PII heurística):", sorted(pii_cols))

SAL = "fatesg-ia2-salt-2025"
def hash_value(x: str) -> str:
    h = hashlib.sha256((SAL + str(x)).encode('utf-8')).hexdigest()
    return h[:12]

df_anon = df_diarias.copy()
for col in pii_cols:
    df_anon[col] = df_anon[col].astype(str).apply(hash_value)

saida = Path('diarias_anonimizadas.csv')
df_anon.to_csv(saida, index=False)
print(f"\nArquivo anonimizado salvo em: {saida.resolve()}")

print("\nTop colunas categóricas por cardinalidade (até 10):")
cat_cols = df_anon.select_dtypes(include=['object']).columns
card = df_anon[cat_cols].nunique().sort_values(ascending=False).head(10)
print(card)

print("\nExemplo de distribuição de uma coluna categórica (a maior cardinalidade):")
if len(card):
    col0 = card.index[0]
    print(df_anon[col0].value_counts().head(10))

Dimensões: (96574, 9)
Colunas: ['CARGOSERVIDOR', 'DATAPARTIDA', 'DATARETORNO', 'DESTINOVIAGEM', 'NOMESERVIDOR', 'SIGLAORGAO', 'VALORTOTAL', 'ANOMES', 'CARGOSERVIDOR_PADRONIZADO']

Tipos de dados:
CARGOSERVIDOR                 object
DATAPARTIDA                   object
DATARETORNO                   object
DESTINOVIAGEM                 object
NOMESERVIDOR                  object
SIGLAORGAO                    object
VALORTOTAL                   float64
ANOMES                         int64
CARGOSERVIDOR_PADRONIZADO      int64
dtype: object

Valores ausentes por coluna (top 20):
CARGOSERVIDOR                0
DATAPARTIDA                  0
DATARETORNO                  0
DESTINOVIAGEM                0
NOMESERVIDOR                 0
SIGLAORGAO                   0
VALORTOTAL                   0
ANOMES                       0
CARGOSERVIDOR_PADRONIZADO    0
dtype: int64

Estatísticas descritivas (numéricas):
                             count           mean         std        min  \
VALORTOTAL 